# Phân tích PCA để nhận diện khuôn mặt

### Bước 1: Cài đặt thư viện

In [1]:
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pickle

### Bước 2: Đọc ảnh và tiền xử lý dữ liệu



In [12]:
DATASET_DIR = 'dataset'
IMG_SIZE = (100, 100)  # Resize về 100x100

X, y = [], []
label_map = {}

image_shapes = []

for idx, student in enumerate(os.listdir(DATASET_DIR)):
    label_map[idx] = student
    folder = os.path.join(DATASET_DIR, student)
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        image_shapes.append(img.shape)  # Lưu lại kích thước gốc
        
        img = cv2.resize(img, IMG_SIZE)
        X.append(img.flatten())
        y.append(idx)

X = np.array(X)
y = np.array(y)

print(f"Tổng số sinh viên: {len(label_map)}")
print(f"Tổng số ảnh đã nạp: {len(X)}")
print(f"Kích thước gốc (chỉ cần xét ảnh đầu tiên): {image_shapes[0] if image_shapes else 'Không có ảnh'}")
print(f"Kích thước sau xử lý: {IMG_SIZE}")

Tổng số sinh viên: 6
Tổng số ảnh đã nạp: 6
Kích thước gốc (chỉ cần xét ảnh đầu tiên): (250, 200)
Kích thước sau xử lý: (100, 100)


### Bước 3: Huấn luyện PCA và lưu model

In [14]:
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X)

# Lưu model
with open('model.pkl', 'wb') as f:
    pickle.dump((pca, X_pca, y, label_map), f)

print("Đã huấn luyện PCA và lưu model thành công!")


Đã huấn luyện PCA và lưu model thành công!


### Bước 4: Nhận diện khuôn mặt từ ảnh mới

In [23]:
# Tải model
with open('model.pkl', 'rb') as f:
    pca, X_pca, y, label_map = pickle.load(f)

# Nhận diện ảnh test
test_img = cv2.imread('test_face_1a.png', cv2.IMREAD_GRAYSCALE)
test_img = cv2.resize(test_img, IMG_SIZE)
test_flat = test_img.flatten()
test_pca = pca.transform([test_flat])

# So sánh
distances = np.linalg.norm(X_pca - test_pca, axis=1)
min_index = np.argmin(distances)
predicted_id = y[min_index]
predicted_name = label_map[predicted_id]

print(f"Ảnh test được nhận diện là: {predicted_name}")

Ảnh test được nhận diện là: studentA
